In [59]:
# %pip install /home/vladiouz/MvX/mx-sdk-py
# %pip install ipytest
# %pip install asyncio
import time

In [47]:
from multiversx_sdk import Transaction, TransactionComputer, UserSigner, ApiNetworkProvider, Address
from pathlib import Path
import ipytest

ipytest.autoconfig()

provider = ApiNetworkProvider("https://devnet-api.multiversx.com")
signer_owner = UserSigner.from_pem_file(Path("/home/vladiouz/MvX/mx-contracts-rs/contracts/mvx-game-sc/interactor/wallet1.pem"))
signer_second_user = UserSigner.from_pem_file(Path("/home/vladiouz/MvX/mx-contracts-rs/contracts/mvx-game-sc/interactor/wallet2.pem"))
signer_third_user = UserSigner.from_pem_file(Path("/home/vladiouz/MvX/mx-contracts-rs/contracts/mvx-game-sc/interactor/wallet3.pem"))
owner_address = Address.from_bech32("erd1r6f7nfpyzul2tef7gne5h6nx9xqnyt5gehwltlxymnqkztjjzvuqdhderc")
second_user_address = Address.from_bech32("erd1tjusdv806tuwzllgesljglm7y9jef38wdylkvp85v7a46z9x23us0z5xtr")
third_user_address = Address.from_bech32("erd1pu4r9rxgn8f7a7gwjchtxjz6y4u3ha7fy93w6r3fjeq26jaqkqjs4ly8fd")
contract = Address.from_bech32("erd1qqqqqqqqqqqqqpgq22z56q99vurkc7aqlv5ds2gfs823umngzvuqnyrmzj")
token_id = "VLD-76ecd8"
another_token_id = "WRNG-67975d"
third_token_id = "RAND-e3641c"
fee_amount = 1
wage_amount = 1
waiting_time = 10000000000000

#### setup function calls

In [36]:
from multiversx_sdk.abi import Abi
from multiversx_sdk import SmartContractTransactionsFactory, TransactionsFactoryConfig
from multiversx_sdk import QueryRunnerAdapter, SmartContractQueriesController

abi = Abi.load("mvx-game-sc.abi.json")
config = TransactionsFactoryConfig(chain_id="D")
factory = SmartContractTransactionsFactory(config, abi)
query_runner = QueryRunnerAdapter(provider)
query_controller = SmartContractQueriesController(query_runner, abi)

#### `create_game()` EP

In [37]:
from multiversx_sdk import Token, TokenTransfer

def create_game(signer_address, signer, waiting_time, min_players, max_players, wager):
    tx = factory.create_transaction_for_execute(
        sender=signer_address,
        contract=contract,
        function="createGame",
        gas_limit=10000000,
        arguments=[waiting_time, min_players, max_players, wager],
        token_transfers=[TokenTransfer(Token(token_id), fee_amount)]
    )

    transaction_computer = TransactionComputer()

    tx.nonce = provider.get_account(signer_address).nonce
    tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx))
    hash = provider.send_transaction(tx)
    print("Transaction hash:", hash)

#### `join_game()` EP

In [44]:
def join_game(signer_address, signer, game_id):
    tx = factory.create_transaction_for_execute(
        sender=signer_address,
        contract=contract,
        function="joinGame",
        gas_limit=10000000,
        arguments=[game_id],
        token_transfers=[TokenTransfer(Token(token_id), fee_amount)]
    )

    transaction_computer = TransactionComputer()

    tx.nonce = provider.get_account(signer_address).nonce
    tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx))
    hash = provider.send_transaction(tx)
    print("Transaction hash:", hash)

#### `game_start_fee()` view

In [39]:
def game_start_fee():
    return query_controller.query(
        contract=contract.to_bech32(),
        function="getGameStartFee",
        arguments=[],
    )[0]

#### `players()` view

In [40]:
def players(game_id):
    return query_controller.query(
        contract=contract.to_bech32(),
        function="getPlayers",
        arguments=[game_id],
    )[0]

#### `last_game_id()` view

In [41]:
def last_game_id():
    return query_controller.query(
        contract=contract.to_bech32(),
        function="getLastGameId",
        arguments=[],
    )[0]

#### tests

In [71]:
def test_create_game():
    create_game(owner_address, signer_owner, waiting_time, 2, 2, wage_amount)

def test_join_game():
    create_game(owner_address, signer_owner, waiting_time, 2, 2, wage_amount)
    time.sleep(45)
    join_game(second_user_address, signer_second_user, last_game_id())
    time.sleep(45)
    assert players(last_game_id())[0].hex() == second_user_address.to_hex()
    

#### run tests

In [ ]:
ipytest.run()